In [ ]:
from sklearn.metrics import *
import numpy as np
import torch
import torch.nn as nn
from collections import defaultdict
import math

import pandas as pd
import numpy as np
from collections import defaultdict
import math
import re
import gensim
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, auc, f1_score, precision_score, recall_score, roc_auc_score

nltk.download('stopwords')
nltk.download('wordnet')
pd.set_option('display.max_colwidth', -1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
<ipython-input-16-508ef5c754fb>:30: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
"""
Used when run through google colab
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Colab Notebooks/data_preporssing_05042023.csv'
"""

file_path = 'data_preporssing_05042023.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load CSV file containing text and labels
df = pd.read_csv(file_path)

top_50_diag = df['DIAG'].unique()
top_50_dict = {}
num_of_classes=len(df['DIAG'].unique())

for i in range(len(top_50_diag)):
    top_50_dict[top_50_diag[i]] = i

def y_vec(diag):
    vec = np.zeros((50,))
    idx = top_50_dict[diag]
    vec[idx] = 1
    return vec

df["Y_Var"]= df["DIAG"].apply(lambda x: y_vec(x))

# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2)

In [ ]:
# Load pre-trained word2vec model
# saved_word2vec_model = '/content/drive/MyDrive/Colab Notebooks/word2Vec_embeddings_05042023.bin'
saved_word2vec_model = 'word2Vec_embeddings_05042023.bin'
word2vec_model = gensim.models.Word2Vec.load(saved_word2vec_model)

# Set parameters
embedding_dim = word2vec_model.vector_size
hidden_dim = 100
vocab_size = len(word2vec_model.wv)

# Create embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for i, word in enumerate(word2vec_model.wv.index_to_key):
    embedding_matrix[i] = word2vec_model.wv[word]
    

# Tokenize text using word2vec model vocabulary for training set
tokenized_train = train_df['NOTE'].apply(lambda x: [word2vec_model.wv.key_to_index[word] for word in x.split() if word in word2vec_model.wv.key_to_index])

# Tokenize text using word2vec model vocabulary for testing set
tokenized_test = test_df['NOTE'].apply(lambda x: [word2vec_model.wv.key_to_index[word] for word in x.split() if word in word2vec_model.wv.key_to_index])

# Set maximum sequence length
max_seq_length = 400

# Pad sequences to the same length for training set
padded_train = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(i[:max_seq_length]) for i in tokenized_train], batch_first=True)

# Pad sequences to the same length for testing set
padded_test = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(i[:max_seq_length]) for i in tokenized_test], batch_first=True)


In [ ]:
X_train=padded_train
X_test=padded_test
Y_train = torch.from_numpy(np.asarray(train_df['Y_Var'].tolist(), dtype=np.float32))
Y_test = torch.from_numpy(np.asarray(test_df['Y_Var'].tolist(), dtype=np.float32))



In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_size=100,  num_layers=2, dropout=0.8, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2 ,  num_of_classes)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        #print(x.shape)
        embedded = self.embedding(x)
        #print(embedded.shape)
        output, (hidden_states, cell_states) = self.lstm(embedded)
        #print(output.shape)
        output = self.fc(output)
        #print(output.shape)
        output = torch.sum(output, dim=1)
        #print(output.shape)
        return self.sigmoid(output)


model = Net()

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
criterion = None
optimizer = None

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def classification_metrics(Y_score, Y_pred, Y_true):
    acc, auc, precision, recall, f1score = accuracy_score(Y_true, Y_pred), \
                                           roc_auc_score(Y_true, Y_score), \
                                           precision_score(Y_true, Y_pred), \
                                           recall_score(Y_true, Y_pred), \
                                           f1_score(Y_true, Y_pred)
    return acc, auc, precision, recall, f1score

In [ ]:
def evaluate(model, X, Y):
    model.eval()
    all_y_true = torch.LongTensor()
    all_y_pred = torch.LongTensor()
    all_y_score = torch.FloatTensor()
    y_hat = model(X)
    for i in range(len(X)):
        y_pred = (y_hat[i] > 0.2).int()
        all_y_true = torch.cat((all_y_true, Y[i].to('cpu')), dim=0)
        all_y_pred = torch.cat((all_y_pred,  y_pred.to('cpu')), dim=0)
        all_y_score = torch.cat((all_y_score,  y_hat[i].to('cpu')), dim=0)
        
    acc, auc, precision, recall, f1 = classification_metrics(all_y_score.detach().numpy(), 
                                                              all_y_pred.detach().numpy(), 
                                                              all_y_true.detach().numpy())
    print(f"acc: {acc:.3f}, auc: {auc:.3f}, precision: {precision:.3f}, recall: {recall:.3f}, f1: {f1:.3f}")
    return

In [ ]:
print("model perfomance before training:")
#evaluate(model, X_train, Y_train)
evaluate(model, X_test, Y_test)

model perfomance before training:
acc: 0.496, auc: 0.458, precision: 0.018, recall: 0.458, f1: 0.035


In [ ]:
# prep model for training
model.train()
   
# Set parameters
batch_size = 400
num_epochs = 50

# Train the model
for epoch in range(num_epochs):
    train_loss = 0
    for i in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        batch_train = X_train[i:i+batch_size]
        batch_labels = Y_train[i:i+batch_size]
        output_train = model(batch_train)
        loss = criterion(output_train, batch_labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        #print(f"Epoch: {epoch+1}/{num_epochs}, Batch: {i//batch_size+1}/{len(X_train)//batch_size+1}, Training Loss: {loss.item():.4f}")
    
    train_loss = train_loss / len(X_train)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))
    print("Test Accuracy:")
    evaluate(model, X_test, Y_test)

    

Epoch: 1 	Training Loss: 0.015927
Test Accuracy:
acc: 0.918, auc: 0.619, precision: 0.039, recall: 0.133, f1: 0.060
Epoch: 2 	Training Loss: 0.010672
Test Accuracy:
acc: 0.934, auc: 0.684, precision: 0.048, recall: 0.124, f1: 0.070
Epoch: 3 	Training Loss: 0.010526
Test Accuracy:
acc: 0.937, auc: 0.713, precision: 0.069, recall: 0.174, f1: 0.099
Epoch: 4 	Training Loss: 0.010424
Test Accuracy:
acc: 0.939, auc: 0.746, precision: 0.088, recall: 0.221, f1: 0.126
Epoch: 5 	Training Loss: 0.010316
Test Accuracy:
acc: 0.939, auc: 0.810, precision: 0.106, recall: 0.272, f1: 0.152
Epoch: 6 	Training Loss: 0.010249
Test Accuracy:
acc: 0.940, auc: 0.831, precision: 0.119, recall: 0.313, f1: 0.173
Epoch: 7 	Training Loss: 0.010214
Test Accuracy:
acc: 0.939, auc: 0.848, precision: 0.135, recall: 0.375, f1: 0.199
Epoch: 8 	Training Loss: 0.010155
Test Accuracy:
acc: 0.939, auc: 0.870, precision: 0.144, recall: 0.413, f1: 0.213
Epoch: 9 	Training Loss: 0.009226
Test Accuracy:
acc: 0.959, auc: 0.871,

In [ ]:
#saved_model_path = '/content/drive/MyDrive/Colab Notebooks/Word2Vec.pth'
saved_model_path = 'Word2Vec.pth'
torch.save(model.state_dict(), saved_model_path)